In [1]:
import pennylane.numpy as np
from scipy.linalg import null_space, eigvals, expm
import pennylane as qml
from itertools import combinations
from classical_betti_calc import boundary, homology, betti
from utils import gershgorin, find_cliques

In [2]:
np.random.seed(0)
np.set_printoptions(precision=4)

In [3]:
# Create random edges
edges = set(map("".join, combinations("0123456789", 2)))  # Vertex labels -> Number of vertices
for _ in range(int(len(edges) * 0.3)):  # Remove 30% of edges
    edges.pop()

In [4]:
# Generate a simplicial complex
sxs = find_cliques(edges)
sxs1 = set()
for u in sxs[1:]:
    sxs1 = sxs1.union(u)
sc = list(sxs1)

In [5]:
# Randomly remove some simplicies to increase betti number values
# This might not technically fulfill all conditions for a simplicial complex but works as demonstration
np.random.shuffle(sc)
sc = sc[: int(len(sc) * 0.3)]
sc.extend(list(sxs[1]))
sc = list(set(sc))
sc.sort()

In [6]:
# Classically calculate betti numbers
bnd, simplicies = boundary(sc)  # Boundary operators
Homo = homology(bnd)
b = betti(Homo)  # Betti numbers
print("Betti Numbers:", b)

Betti Numbers: [2, 0, 4, 2, 0]


In [7]:
# Quantum calculation for k-th betti number
k = 0
# Min and max precision qubits to use for QPE
min_pre = 1
max_pre = 10

In [8]:
# Calculate the combinatorial laplacian and verify betti number classically
comb_lap = bnd[k].T @ bnd[k] + bnd[k + 1] @ bnd[k + 1].T
kernel_cp = null_space(comb_lap)
betti_k_dim = kernel_cp.shape[1]
# This is the value we want to estimate using quantum computing
betti_k_zero_eig = np.count_nonzero(np.absolute(eigvals(comb_lap)) < 10**-7)
print(f"Betti number {k}:", betti_k_dim, betti_k_zero_eig)
num = int(np.ceil(np.log2(comb_lap.shape[0])))

Betti number 0: 2 2


In [9]:
# Estimate max eigenvalue
max_eig = max(gershgorin(comb_lap), 1)

In [10]:
# Unitary generation for QPE
# Padding to make size a power of 2
# Scaling to get eigenvalues in [0,1)
H = np.identity(2**num) * max_eig / 2
H[: comb_lap.shape[0], : comb_lap.shape[0]] = comb_lap
H = H / max_eig * 6
U = expm(1j * H)

In [11]:
print(
    "Number of precision qubits",
    "|",
    "Estimated betti number with respective shots",
    "|",
    "Absolute error",
)
for pre in range(min_pre, max_pre + 1):
    dev = qml.device(
        "lightning.qubit",
        wires=2 * num + pre,
        shots=[10**1, 10**2, 10**3, 10**4, 10**5, 10**6],
    )

    @qml.qnode(dev)
    def circ():
        for i in range(num):
            qml.Hadamard(num + i)
        for i in range(num):
            qml.CNOT([num + i, i])
        qml.QuantumPhaseEstimation(
            U,
            target_wires=range(num, 2 * num),
            estimation_wires=range(2 * num, 2 * num + pre),
        )
        return qml.probs(range(2 * num, 2 * num + pre))

    probs = circ()
    print(
        pre,
        probs[:, 0] * 2**num,
        np.abs((betti_k_zero_eig - probs[:, 0] * 2**num)),
    )

Number of precision qubits | Estimated betti number with respective shots | Absolute error
1 [8.     8.16   7.088  7.3168 7.3922 7.3898] [6.     6.16   5.088  5.3168 5.3922 5.3898]
2 [1.6    3.2    4.96   5.28   5.2458 5.2461] [0.4    1.2    2.96   3.28   3.2458 3.2461]
3 [4.8    3.36   3.168  3.4624 3.4026 3.4268] [2.8    1.36   1.168  1.4624 1.4026 1.4268]
4 [0.     1.92   2.448  2.296  2.289  2.3218] [2.     0.08   0.448  0.296  0.289  0.3218]
5 [1.6    1.6    2.112  2.0512 2.0922 2.0956] [0.4    0.4    0.112  0.0512 0.0922 0.0956]
6 [1.6    2.88   2.08   1.944  2.0072 2.0146] [0.4    0.88   0.08   0.056  0.0072 0.0146]
7 [3.2    2.56   1.968  1.8896 2.021  2.0019] [1.2    0.56   0.032  0.1104 0.021  0.0019]
8 [0.     0.64   2.112  2.0512 2.0165 1.996 ] [2.     1.36   0.112  0.0512 0.0165 0.004 ]
9 [0.     1.6    2.16   2.064  2.0078 2.0053] [2.     0.4    0.16   0.064  0.0078 0.0053]
10 [1.6    2.4    2.128  1.9728 2.0062 2.0059] [0.4    0.4    0.128  0.0272 0.0062 0.0059]


In [12]:
print("Quantum Circuit")
print(
    qml.draw(
        circ,
        show_all_wires=True,
        show_matrices=False,
        max_length=10000,
        expansion_strategy="device",
    )()
)


Quantum Circuit
 0: ────╭X──────────────────────────────────────────────────────────────────────────────────────┤       
 1: ────│──╭X───────────────────────────────────────────────────────────────────────────────────┤       
 2: ────│──│──╭X────────────────────────────────────────────────────────────────────────────────┤       
 3: ────│──│──│──╭X─────────────────────────────────────────────────────────────────────────────┤       
 4: ──H─╰●─│──│──│──╭U(M0)─╭U(M1)─╭U(M2)─╭U(M3)─╭U(M4)─╭U(M5)─╭U(M6)─╭U(M7)─╭U(M8)─╭U(M9)───────┤       
 5: ──H────╰●─│──│──├U(M0)─├U(M1)─├U(M2)─├U(M3)─├U(M4)─├U(M5)─├U(M6)─├U(M7)─├U(M8)─├U(M9)───────┤       
 6: ──H───────╰●─│──├U(M0)─├U(M1)─├U(M2)─├U(M3)─├U(M4)─├U(M5)─├U(M6)─├U(M7)─├U(M8)─├U(M9)───────┤       
 7: ──H──────────╰●─├U(M0)─├U(M1)─├U(M2)─├U(M3)─├U(M4)─├U(M5)─├U(M6)─├U(M7)─├U(M8)─├U(M9)───────┤       
 8: ──H─────────────╰●─────│──────│──────│──────│──────│──────│──────│──────│──────│──────╭QFT†─┤ ╭Probs
 9: ──H────────────────────╰●─────│────